In [1]:
###: Importing H2O 

In [2]:
import h2o

In [3]:
###: Initalizing H2O cluster

In [4]:
h2o.init()

ERROR:h2o:Key init.version_check is not a valid config key


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 days 10 hours 17 mins
H2O cluster version:,3.14.0.7
H2O cluster version age:,1 month and 6 days
H2O cluster name:,H2O_started_from_R_avkashchauhan_ppg732
H2O cluster total nodes:,1
H2O cluster free memory:,2.974 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [5]:
###: Importing craigslist jobs title and description dataset into H2O cluster memory

In [6]:
jobs_data= h2o.import_file(col_names=['category', 'jobtitle'], header=1, col_types = ["enum", "string"],
                        path="https://raw.githubusercontent.com/h2oai/sparkling-water/rel-1.6/examples/smalldata/craigslistJobTitles.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
###: Understanding the dataset

In [8]:
jobs_data.summary()

,category,jobtitle
type,enum,string
mins,,NaN
mean,,NaN
maxs,,NaN
sigma,,NaN
zeros,,0
missing,0,0
0,education,After School Supervisor
1,education,"*****TUTORS NEEDED - FOR ALL SUBJECTS, ALL AGES*****"
2,education,Bay Area Family Recruiter


In [9]:
###: Definition of stop words which will be removed from the list of words in out job details

In [10]:
STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can","lines","re","what",
               "there","all","we","one","the","a","an","of","or","in","for","by","on",
               "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
               "from","at","my","be","by","not","that","to","from","com","org","like","likes","so"]

In [11]:
###: Local Tokenize function to tokenize all the words in a list

In [12]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [13]:
###: Calling the tokenize function to tokenize all the words in the job details 

In [14]:
##words_token = local_tokenize(all_sentences)
words_token = tokenize(jobs_data['jobtitle'])

In [15]:
###: Getting total number of tokens

In [16]:
print(len(words_token))

78924


In [17]:
###: The tokens type

In [18]:
type(words_token)

h2o.frame.H2OFrame

In [19]:
###: Understanding Words Token Data as H2O Dataframe

In [20]:
words_token.describe()

Rows:78924
Cols:1




,C1
type,string
mins,NaN
mean,NaN
maxs,NaN
sigma,NaN
zeros,0
missing,13845
0,after
1,school
2,supervisor


In [21]:
###: Importing H2O Word2VEC Estimator

In [22]:
from h2o.estimators.word2vec import H2OWord2vecEstimator

In [23]:
###: Setting Word2Vec estimator configuration

In [24]:
h2o_w2v_model =  H2OWord2vecEstimator(sent_sample_rate = .9, epochs = 10, model_id = "h2o_w2v_model_python")

In [25]:
###: Training Word2Vec Model

In [26]:
h2o_w2v_model.train(training_frame = words_token)

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [27]:
###: Understanding Word2Vec Model

In [28]:
h2o_w2v_model

Model Details
H2OWord2vecEstimator :  Word2Vec
Model Key:  h2o_w2v_model_python
No model summary for this model



In [29]:
###: Testing simlar word Experiment from the model with the word "money"

In [30]:
h2o_w2v_model.find_synonyms( "money", count = 5)

OrderedDict([(u'rides', 0.7279952764511108),
             (u'fares', 0.7047621011734009),
             (u'more', 0.6669892072677612),
             (u'enjoy', 0.5241783857345581),
             (u'judgment', 0.4911399185657501)])

In [31]:
###: Testing simlar word Experiment from the model with the word "account"

In [32]:
h2o_w2v_model.find_synonyms( "account", count = 5)

OrderedDict([(u'liaison', 0.5699155330657959),
             (u'rockstar', 0.5654222369194031),
             (u'dispatch', 0.565384566783905),
             (u'emergency', 0.5629708170890808),
             (u'relations', 0.5588938593864441)])

In [33]:
###: Calculating vector for every job title by passing original job title category data 

In [34]:
job_title_vecs = h2o_w2v_model.transform(words_token, aggregate_method = "AVERAGE")

In [35]:
job_title_vecs.shape

(13845, 100)

In [36]:
job_title_vecs

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
0.0179551,-0.00797802,0.0815321,0.349933,-0.0178768,0.0500604,0.318329,-0.159813,0.277998,-0.219391,0.192949,0.297739,0.0722817,-0.0560711,0.153139,-0.112632,0.192903,0.253662,0.103432,0.212026,-0.175511,0.192271,-0.172411,0.0183183,0.186244,-0.29612,0.307857,0.0347665,0.110348,-0.0591083,-0.296392,0.309013,-0.290719,-0.146808,-0.0632934,-0.19099,0.126214,-0.0523888,0.0522938,0.0643614,0.0384917,0.34891,-0.134788,-0.0729285,-0.314501,-0.0133686,-0.0478583,-0.20319,-0.413155,0.258811,-0.0735133,-0.0568653,-0.241278,0.314083,0.328764,-0.223345,0.257641,-0.247055,-0.0732505,-0.217212,0.496074,0.0379671,-0.0622384,0.150624,0.40383,-0.212764,0.00386726,-0.226638,0.175959,0.409737,0.0130717,0.105451,0.375842,0.078988,-0.0604978,0.0613854,0.285663,-0.21703,0.317409,0.257375,-0.158201,0.256061,-0.19042,0.137072,-0.213676,0.184386,0.177599,-0.463842,-0.270951,-0.201231,-0.282052,0.0816493,0.305745,-0.164701,0.0476012,0.128511,-0.19867,0.270144,-0.158304,-0.191211
0.198496,0.245932,0.182466,0.590663,0.530774,-0.389897,-0.00173687,-0.272299,-0.0296782,-0.131025,-0.140057,0.300605,-0.562641,-0.139242,-0.0978478,0.195237,0.249979,-0.147094,-0.141088,0.0572662,-0.0697324,-0.414618,0.0451773,0.0123859,-0.136952,-0.0700904,0.250318,0.191079,0.180666,0.0583874,-0.376978,0.0461544,-0.699312,0.173983,0.0955267,0.0935854,0.167072,-0.101795,0.116582,0.291421,0.204701,0.126056,0.400053,-0.0490436,-0.174219,0.448838,0.261596,0.0429197,-0.551901,-0.326154,-0.106464,-0.144055,0.0895844,0.395096,-0.248494,-0.207155,-0.0530967,-0.396468,0.289681,-0.164359,0.462974,0.13167,-0.45906,-0.322112,-0.0669346,-0.0275884,-0.00550211,-0.00746805,-0.102739,0.505109,0.0656397,0.295207,0.541028,-0.552472,0.239445,0.204306,0.0301934,-0.456681,0.322662,-0.00547394,0.321132,-0.265258,-0.335424,0.366437,-0.0687196,0.143206,-0.299311,0.0606533,-0.096279,-0.22924,-0.0533951,-0.108387,0.248719,0.114425,0.670337,-0.462584,-0.108296,-0.0541149,0.533516,0.135194
-0.279512,-0.0311573,0.0299129,0.139332,-0.188786,-0.162513,0.138589,0.0292393,0.210391,-0.0854794,-0.151968,0.352379,-0.0874313,-0.238089,-0.0754317,0.280004,-0.135037,-0.0122419,0.224876,-0.101882,-0.184141,-0.187821,0.167805,-0.0573033,-0.0675215,-0.115679,-0.000559647,0.221218,-0.0624837,0.367284,-0.288968,0.0154089,-0.0160931,0.054836,-0.156388,0.0201588,-0.122521,-0.116863,-0.198888,0.210559,0.114788,0.0229043,0.0972436,-0.170387,0.0957396,0.154114,-0.00425321,0.0699815,-0.177799,-0.121043,0.132781,-0.182174,-0.139764,0.00117804,-0.0180625,-0.121897,-0.218152,-0.373024,0.370585,-0.0274652,-0.154936,0.248075,0.0508673,-0.114537,0.00839755,-0.0361308,0.180554,0.21724,-0.0615257,-0.19364,0.0877388,-0.164708,0.155988,-0.220329,0.168677,0.0529324,0.163873,-0.135629,0.214686,0.0698141,-0.0667781,0.076631,0.0362696,0.383407,-0.0370228,-0.0298729,0.116152,0.056563,-0.0529871,-0.101769,0.0833967,-0.0436793,-0.176527,0.140998,0.141348,0.127693,-0.0852296,0.216663,-0.0940938,-0.130074
-0.18584,0.0589173,-0.0209236,0.32223,-0.328547,-0.0292025,0.141208,-0.113125,0.42417,0.123439,0.00357873,0.0485908,-0.169337,-0.160606,-0.0947159,-0.00983132,-0.224349,0.105742,-0.2903,0.158163,0.173661,0.115932,0.0255834,-0.0720259,0.0558388,-0.126619,0.122635,0.0349655,-0.156523,0.0614013,-0.267123,0.0665683,-0.0753392,-0.187512,-0.210097,-0.236664,-0.0322936,0.0496863,0.169302,-0.0962571,0.232475,0.180473,0.0783525,0.0707957,-0.118951,0.122508,0.0507294,0.19402,-0.179745,0.0475672,-0.0552222,-0.142431,-0.0151779,-0.104452,-0.0661279,0.0224244,0.0312314,-0.375251,0.245219,-0.142491,0.0860295,0.0130573,0.165486,0.0456134,0.0480496,0.00788409,0.248232,0.081

In [37]:
jobs_data.shape

(13845, 2)

In [38]:
jobs_data['category'].shape

(13845, 1)

In [39]:
###: Checking if any of the job title vectors are empty or NAs

In [40]:
valid_job_titles = ~ job_title_vecs["C1"].isna()
valid_job_titles

not(isNA(C1))
1
1
1
1
1
1
1
1
1
1


In [41]:
###: Combining categories with valid job data to generate the final data set which will be used to build H2O GBM Model

In [42]:
#final_words_data = jobs_data['category'].cbind(job_title_vecs)
final_words_data = jobs_data[valid_job_titles,:].cbind(job_title_vecs[valid_job_titles,:])

In [43]:
###: Creating training, validation & test dataset from the source dataset

In [44]:
train, valid, test = final_words_data.split_frame(ratios=[0.8, 0.1])

In [45]:
###: Setting the response column

In [46]:
response = "category"

In [47]:
###: Setting the response column as enum/categorical in all train, valid and test dataset

In [48]:
train[response] = train[response].asfactor()
valid[response] = valid[response].asfactor()
test[response] = test[response].asfactor()

In [49]:
###: Setting features columns list for training

In [50]:
features = train.col_names
features.remove(response)

In [51]:
###: Importing H2O GBM Estimator

In [52]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [53]:
###: Setting H2O GBM configuration

In [54]:
h2o_gbm = H2OGradientBoostingEstimator(nfolds = 5)

In [55]:
###: Traning the H2O GBM model based on jobs data

In [56]:
h2o_gbm.train(x = features, y = response, training_frame= train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [57]:
###: H2O GBM Model performance

In [58]:
h2o_gbm.model_performance(xval=True)


ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.182693711286
RMSE: 0.427426849046
LogLoss: 0.577215565131
Mean Per-Class Error: 0.204715957269
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



accounting,administrative,customerservice,education,foodbeverage,labor,Error,Rate
1005.0,167.0,52.0,10.0,9.0,14.0,0.2004773,"252 / 1,257"
96.0,1568.0,202.0,45.0,31.0,40.0,0.2088799,"414 / 1,982"
38.0,202.0,1253.0,50.0,121.0,185.0,0.3223364,"596 / 1,849"
11.0,65.0,45.0,1797.0,18.0,15.0,0.0789339,"154 / 1,951"
6.0,55.0,119.0,18.0,1598.0,199.0,0.1989975,"397 / 1,995"
15.0,49.0,162.0,33.0,172.0,1540.0,0.2186707,"431 / 1,971"
1171.0,2106.0,1833.0,1953.0,1949.0,1993.0,0.2039073,"2,244 / 11,005"


Top-6 Hit Ratios: 


k,hit_ratio
1,0.7960927
2,0.9186733
3,0.9664698
4,0.9868242
5,0.9963652
6,0.9999999


In [59]:
###: Performing the prediction with the test dataset

In [60]:
pred_result = h2o_gbm.predict(test_data=test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [61]:
###: Checking the prediction results

In [62]:
pred_result

predict,accounting,administrative,customerservice,education,foodbeverage,labor
education,0.0278986,0.124369,0.108486,0.662066,0.0291289,0.048052
education,0.00716205,0.00435998,0.00652709,0.971401,0.00413316,0.00641719
education,0.00201005,0.0112444,0.0104305,0.960422,0.0044671,0.0114257
education,0.00078977,0.00271799,0.00356573,0.989237,0.00164318,0.00204605
education,0.0127883,0.0398004,0.164989,0.647013,0.0677446,0.0676651
education,0.00138501,0.00279026,0.00863425,0.979759,0.00318674,0.00424458
education,0.00863156,0.017872,0.0480161,0.8976,0.0138661,0.0140138
education,0.00102662,0.00178066,0.00468632,0.98468,0.00400643,0.00381952
education,0.00143848,0.0041389,0.00559884,0.981511,0.00244877,0.00486394
administrative,0.100777,0.546673,0.143615,0.144304,0.0306575,0.0339735


In [63]:
###: H2O GBM Model performance on test data

In [64]:
h2o_gbm.model_performance(test_data=test)


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.175580515861
RMSE: 0.419023287969
LogLoss: 0.551114140641
Mean Per-Class Error: 0.199863730921
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



accounting,administrative,customerservice,education,foodbeverage,labor,Error,Rate
126.0,20.0,8.0,2.0,2.0,1.0,0.2075472,33 / 159
16.0,194.0,32.0,8.0,2.0,6.0,0.2480620,64 / 258
4.0,27.0,147.0,4.0,16.0,21.0,0.3287671,72 / 219
2.0,4.0,6.0,233.0,4.0,2.0,0.0717131,18 / 251
0.0,6.0,14.0,4.0,226.0,24.0,0.1751825,48 / 274
2.0,6.0,19.0,2.0,16.0,223.0,0.1679104,45 / 268
150.0,257.0,226.0,253.0,266.0,277.0,0.1959412,"280 / 1,429"


Top-6 Hit Ratios: 


k,hit_ratio
1,0.8040588
2,0.9251225
3,0.9678097
4,0.9846047
5,0.9958013
6,1.0


In [65]:
###: Creating a local function to perform prediction by passing a custom job description

In [66]:
def predict_with_h2o_w2v(job_title,w2v, gbm):
    words = tokenize(h2o.H2OFrame(job_title).ascharacter())
    job_title_vec = w2v.transform(words, aggregate_method="AVERAGE")
    print(gbm.predict(test_data=job_title_vec))

In [67]:
###: Calling the local prediction function to predict category of given job details

In [68]:
predict_local = predict_with_h2o_w2v(["food kitchen sandwich"], h2o_w2v_model, h2o_gbm)
print(predict_local)

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,accounting,administrative,customerservice,education,foodbeverage,labor
foodbeverage,0.000780452,0.001684,0.00915411,0.000943528,0.982074,0.00536387



None


In [69]:
predict_local = predict_with_h2o_w2v(["swimming pool digging in the backyard"], h2o_w2v_model, h2o_gbm)
print(predict_local)

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,accounting,administrative,customerservice,education,foodbeverage,labor
labor,0.00116283,0.00235045,0.0210939,0.00188025,0.0093569,0.964156



None


In [71]:
predict_local = predict_with_h2o_w2v(["programming in Javascript and web", "personal assistance to the doctor"], h2o_w2v_model, h2o_gbm)
print(predict_local)

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,accounting,administrative,customerservice,education,foodbeverage,labor
education,0.0592295,0.121967,0.157455,0.539883,0.0467595,0.0747068
administrative,0.0451113,0.835459,0.077937,0.00478149,0.0151103,0.0216009



None
